## Running NIDN

We are now ready to get started.
Before we go ahead, we must do the imports.

### Imports 

In [ ]:
### Imports (TODO remove this when finished)
%load_ext autoreload
%autoreload 2

# Append root folder in case you haven't installed NIDN
import sys
sys.path.append("../")

import nidn

In [ ]:
# Load default cfg as starting point
cfg = nidn.load_default_cfg()

# Specify your desired range of wavelengths
cfg.physical_wavelength_range[0] = (1.3)*1e-6
cfg.physical_wavelength_range[1] = (1.7)*1e-6

cfg.N_freq = 40

# Currently, the target spectra is set manually as a list of numbers 
cfg.target_reflectance_spectrum =   [0.99274957,0.99396029,0.99476812,0.99529796,0.99562242,0.99578140,0.99579159,0.99564959,0.99532948,0.99477284,0.99386466,0.99237639,0.98981972,0.98501409,0.97450288,0.94444840,0.79571011,0.04614844,0.85296015,0.95379278,0.97740940,0.98624919,0.99044192,0.99272103,0.99406357,0.99488566,0.99538570,0.99566431,0.99577208,0.99572991,0.99553712,0.99517277,0.99459136,0.99371174,0.99239463,0.99039858,0.98729128,0.98226215,0.97370298,0.95820717,]
cfg.target_transmittance_spectrum = [0.00707325,0.00586522,0.00505880,0.00452937,0.00420452,0.00404450,0.00403269,0.00417247,0.00448975,0.00504278,0.00594624,0.00742800,0.00997487,0.01476375,0.02524040,0.05519899,0.20346735,0.95065199,0.14638822,0.04587472,0.02233243,0.01352009,0.00934008,0.00706765,0.00572883,0.00490882,0.00440979,0.00413141,0.00402320,0.00406428,0.00425526,0.00461696,0.00519465,0.00606919,0.00737939,0.00936598,0.01246021,0.01747083,0.02600304,0.04145794,]
# Since R + T + A = 1, we only need to give the reflectance and transmittance (absorptance is implicit)

nidn.plot_spectrum(cfg,
                   cfg.target_reflectance_spectrum,
                   cfg.target_transmittance_spectrum)

physical_wls, normalized_freqs = nidn.get_frequency_points(cfg)
print("Physical wavelengths are (in meters):")
print(physical_wls)

## Example 1 - Uniform single-layer with unrestricted epsilon

Let's start with a uniform single-layer and see if NIDN can get sufficiently close to the ground truth.

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 1 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 1
cfg.N_layers = 25 # Choose number of layers

cfg.TRCWA_PER_LAYER_THICKNESS = [0.25,0.1704545] * 6 + [0.5] + [0.1704545,0.25] * 6

# Allowed range of epsilon values
cfg.real_min_eps = 0.0
cfg.real_max_eps = 20.0
cfg.imag_min_eps = 0.0
cfg.imag_max_eps = 3.0

cfg.type = "classification" # Choose type as described above
cfg.iterations = 1000 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
#Show all used settings
nidn.print_cfg(cfg)

`print_cfg(cfg)` shows you more or less everything you want to know about the config.
Using `run_training(cfg)`, we run the network until it reaches the number of iterations set above (or until you interrupt it).

In [ ]:
cfg.iterations = 2000
nidn.run_training(cfg);

### Interpretation of results

#### Loss plot

The loss as a function of model evaluations is presented below. As the training evolves, the three losses here, [L1](https://afteracademy.com/blog/what-are-l1-and-l2-loss-functions), Loss, and Weighted Average Loss, can be seen to decrease. {Pablo add link/explanation to the other losses}.

In [ ]:
nidn.plot_losses(cfg)

#### Spectrum plots

The produced RTA spectra are plotted together with the target spectra in the figure below.

In [ ]:
nidn.plot_spectra(cfg)

#### Absolute grid values plot

The complex absolute value of the epsilon over all frequencies is presented here. This plot is in general more useful for patterned multilayers.

In [ ]:
nidn.plot_model_grid(cfg)

#### Epsilon vs frequency and real materials

The following function plots the epsilon values vs. frequency of grid points against real materials in our library. This plot is in general more useful for patterned multilayers.

In [ ]:
nidn.plot_eps_per_point(cfg)

## Example 2 - Uniform single-layer with materials classification

Next up is the same example, a uniform single-layer of titanium oxide, but this time we check if NIDN can predict the correct material.

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 1 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 1
cfg.N_layers = 3 # Choose number of layers
cfg.use_regularization_loss = False

cfg.type = "classification" # Choose type as described above
cfg.iterations = 1000 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)

As can be seen from the plots, the prediction is correct and the loss is even lower.

## Example 3 - Optical filter with regression

The goal of using NIDN is to find a structure with reflection, transmission, and reflection spectra as close to you target spectra as possible. We might for instance want a filter that has a high transmission for wavelengths around 1550 nm and low transmission for other wavelengths. The reflection should be opposite to that of the transmission and the absorption should be minimal for all wavelengths.
The target spectra with these requirements can be set using the following code:

For the third and final example, we will try to find a structure that satisfies these requirements setting the target spectra using regression. The structure would work as an optical filter with transmission only for wavelengths around 1550 nm.
The structure consists of Y layers with X x X grid points per layer. As can be seen in the code below, we make use of oversampling.

In [ ]:
# Define target for the filter
cfg = nidn.load_default_cfg()
cfg.N_freq = 20
cfg.target_reflectance_spectrum =   7*[1.0] + 1*[0.0] + 12*[1.0]
cfg.target_transmittance_spectrum = 7*[0.0] + 1*[1.0] + 12*[0.0]
cfg.physical_wavelength_range[0] = 1e-5
cfg.physical_wavelength_range[1] = 2e-5
cfg.freq_distribution = "log"


nidn.plot_spectrum(cfg,
                   cfg.target_reflectance_spectrum,
                   cfg.target_transmittance_spectrum)

In [ ]:
# Allowed range of epsilon values
cfg.pop("model",None); # Forget the old model
cfg.real_min_eps = 0.0
cfg.real_max_eps = 20.0
imag_min_eps = 0.0
imag_max_eps = 3.0

cfg.Nx = 9 # Set layer size  to 16x16 (each of the grid points has its own epsilon now)
cfg.Ny = 9
cfg.eps_oversampling = 3
cfg.learning_rate = 3e-5
cfg.N_layers = 3 # Less layer to keep compute managable
cfg.type = "regression" # Choose type as described above (for now still regression)
cfg.iterations = 500 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
# The other plots
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)

NIDN is able to get quite close to the desired spectra with the unrestricted epsilon. 

#### Material ID plot

Finally, we will present another plot, showing the real materials closest to the unrestricted ones for each grid point.
The layers are numbered from bottom to top, and the light is incident on the first layer, i.e., the bottom of the stack.

In [ ]:
nidn.plot_material_grid(cfg)

NB Here we should have plotted the RTA result of this structure.
In case you want to save results you can use this handy function to save it to the results folder with a current timestamp.

In [ ]:
nidn.save_run(cfg)

# You can save all available plots to a single folder using this function
nidn.save_all_plots(cfg,save_path="/results/example/")

In [ ]:
# Squared germanium & tantalum layer
cfg = nidn.load_default_cfg()
cfg.N_freq = 20
cfg.target_reflectance_spectrum =   [0.60248210,0.20808528,0.10825184,0.55990793,0.72659247,0.74876763,0.65817925,0.32626005,0.03080333,0.50414092,0.71607931,0.75605811,0.68509433,0.38961000,0.01012443,0.48532436,0.71720245,0.76067772,0.69521557,0.41321222,]
cfg.target_transmittance_spectrum = [0.38656270,0.77990755,0.88060908,0.43550853,0.27110902,0.24957937,0.33986960,0.67012270,0.96391071,0.49316468,0.28246341,0.24277933,0.31350818,0.60775718,0.98545664,0.51235020,0.28152825,0.23829265,0.30353337,0.58443205,]
cfg.physical_wavelength_range[0] = 2e-6
cfg.physical_wavelength_range[1] = 1e-5
cfg.freq_distribution = "linear"

nidn.plot_spectrum(cfg,
                   cfg.target_reflectance_spectrum,
                   cfg.target_transmittance_spectrum)

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 9 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 9
cfg.eps_oversampling = 3
cfg.N_layers = 1 # Choose number of layers
cfg.type = "regression" # Choose type as described above
cfg.iterations = 100 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)

In [ ]:
cfg.pop("model",None); # Forget the old model
cfg.Nx = 9 # Set layer size  to 1x1 (interpreted as uniform)
cfg.Ny = 9
cfg.eps_oversampling = 3
cfg.N_layers = 1 # Choose number of layers
cfg.type = "classification" # Choose type as described above
cfg.iterations = 100 # Set number of training iterations (that is forward model evaluations) to perform

In [ ]:
nidn.run_training(cfg);

In [ ]:
nidn.plot_losses(cfg)
nidn.plot_spectra(cfg)
nidn.plot_model_grid(cfg)
nidn.plot_eps_per_point(cfg)